In [130]:
import pandas as pd
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import OrdinalEncoder, OneHotEncoder
from sklearn.model_selection import train_test_split
from sklearn.impute import SimpleImputer
import xgboost as xgb
from sklearn.metrics import mean_absolute_error
from sklearn.preprocessing import OrdinalEncoder, OneHotEncoder
import numpy as np
from sklearn.metrics import mean_squared_error

In [139]:
data = pd.read_csv(r'C:\Users\mbauer2\workspace\Uni\enterprise-ai-project\ImmmoWelt_Price_Guide\flats_to_buy_preproceesed.csv')

In [140]:
data

,Url,Title,Object_price,Object_currency,LivingSpace,Rooms,ConstructionYear,ZipCode,EstateTypeGerman,DistributionTypeGerman,...,teppich,terrasse,tiefgarage,vermietet,vollerschlossen,wanne,wasch_trockenraum,wg_geeignet,wintergarten,zentralheizung
0,https://www.immowelt.de/expose/2ar9w55,Ried 67.5 - OPTIMAL,536000.0,EUR,99.34,4.0,NaN,97230,WOHNUNG,ZUM_KAUF,...,0,1,0,0,0,0,0,0,0,0
1,https://www.immowelt.de/expose/2aaaw55,Ried 67.6 - PASSEND,556000.0,EUR,101.21,4.0,NaN,97230,WOHNUNG,ZUM_KAUF,...,0,1,0,0,0,0,0,0,0,0
2,https://www.immowelt.de/expose/2agdg5f,++2- Zi. ETW mit ca. 53m\u00b2 in W\u00fcrzbur...,220000.0,EUR,53.00,2.0,1971.0,97080,WOHNUNG,ZUM_KAUF,...,0,0,0,0,0,1,0,0,0,1
3,https://www.immowelt.de/expose/2ak8w55,Ried 67.1 - IDEAL,645000.0,EUR,117.96,4.0,NaN,97230,WOHNUNG,ZUM_KAUF,...,0,1,0,0,0,0,0,0,0,0
4,https://www.immowelt.de/expose/27zeu5l,"Sehr gepflegte 3Zi-ETW N\u00e4he Uni-Klinik, g...",269000.0,EUR,70.00,3.0,1968.0,97078,WOHNUNG,ZUM_KAUF,...,0,0,1,1,0,1,0,1,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
282,https://www.immowelt.de/expose/2a42q5g,Helle sch\u00f6ne gut geschnittene 75qm ETW Be...,289000.0,EUR,75.00,3.0,2010.0,97084,WOHNUNG,ZUM_KAUF,...,0,0,0,0,0,0,0,0,0,0
283,https://www.immowelt.de/expose/2as2559,"Eigentumswohnung, ca. 76 qm, direkt vom Herste...",541039.0,EUR,76.00,3.0,NaN,97276,WOHNUNG,ZUM_KAUF,...,0,1,0,0,0,0,0,0,0,0
284,https://www.immowelt.de/expose/28bd354,Fern von Stress und Hektik,310000.0,EUR,87.00,4.0,NaN,97084,WOHNUNG,ZUM_KAUF,...,0,0,0,0,0,0,0,0,0,1
285,https://www.immowelt.de/expose/2a76559,"Eigentumswohnung, ca. 66 qm, direkt vom Herste...",474635.0,EUR,66.00,3.0,NaN,97276,WOHNUNG,ZUM_KAUF,...,0,1,0,0,0,0,0,0,0,0


In [141]:
data['ConstructionYear'].isnull().sum()
print(data.isnull().sum())

Url                  0
Title                0
Object_price         8
Object_currency      8
LivingSpace          1
                    ..
wanne                0
wasch_trockenraum    0
wg_geeignet          0
wintergarten         0
zentralheizung       0
Length: 94, dtype: int64


In [142]:
data.dropna(subset=['ConstructionYear'], inplace=True)
data.dropna(subset=['Object_price'], inplace=True)
data.dropna(subset=['Rooms'], inplace=True)
data.drop('Title', axis=1, inplace=True)
data.drop('Object_currency', axis=1, inplace=True)
data.drop('Url', axis=1, inplace=True)
data.drop('EstateType', axis=1, inplace=True)
data.drop('DistributionType', axis=1, inplace=True)
data = data.reindex()

In [143]:
data

,Object_price,LivingSpace,Rooms,ConstructionYear,ZipCode,EstateTypeGerman,DistributionTypeGerman,abstellraum,als_ferienimmobilie_geeignet,altbau_(bis_1945),...,teppich,terrasse,tiefgarage,vermietet,vollerschlossen,wanne,wasch_trockenraum,wg_geeignet,wintergarten,zentralheizung
2,220000.0,53.00,2.0,1971.0,97080,WOHNUNG,ZUM_KAUF,0,0,0,...,0,0,0,0,0,1,0,0,0,1
4,269000.0,70.00,3.0,1968.0,97078,WOHNUNG,ZUM_KAUF,0,0,0,...,0,0,1,1,0,1,0,1,0,1
5,169000.0,33.51,1.0,1983.0,97072,WOHNUNG,ZUM_KAUF,0,0,0,...,0,0,0,0,0,0,1,0,0,0
6,275000.0,65.45,2.0,1958.0,97072,WOHNUNG,ZUM_KAUF,0,0,0,...,0,0,0,0,0,0,0,0,0,0
7,239000.0,79.50,3.0,1977.0,97209,WOHNUNG,ZUM_KAUF,1,0,0,...,0,0,1,0,0,0,0,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
278,349000.0,71.23,2.0,2019.0,97209,WOHNUNG,ZUM_KAUF,0,0,0,...,0,1,0,0,0,0,0,0,0,0
279,95500.0,31.10,1.0,1997.0,97299,WOHNUNG,ZUM_KAUF,1,0,0,...,0,0,0,0,0,0,0,0,0,1
281,240000.0,97.22,4.0,1991.0,97228,WOHNUNG,ZUM_KAUF,0,0,0,...,0,0,0,1,0,1,0,0,0,1
282,289000.0,75.00,3.0,2010.0,97084,WOHNUNG,ZUM_KAUF,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [144]:
print(data.isnull().sum())

Object_price         0
LivingSpace          0
Rooms                0
ConstructionYear     0
ZipCode              0
                    ..
wanne                0
wasch_trockenraum    0
wg_geeignet          0
wintergarten         0
zentralheizung       0
Length: 89, dtype: int64


In [146]:
data = data.reset_index(drop=True)

In [147]:
data

,Object_price,LivingSpace,Rooms,ConstructionYear,ZipCode,EstateTypeGerman,DistributionTypeGerman,abstellraum,als_ferienimmobilie_geeignet,altbau_(bis_1945),...,teppich,terrasse,tiefgarage,vermietet,vollerschlossen,wanne,wasch_trockenraum,wg_geeignet,wintergarten,zentralheizung
0,220000.0,53.00,2.0,1971.0,97080,WOHNUNG,ZUM_KAUF,0,0,0,...,0,0,0,0,0,1,0,0,0,1
1,269000.0,70.00,3.0,1968.0,97078,WOHNUNG,ZUM_KAUF,0,0,0,...,0,0,1,1,0,1,0,1,0,1
2,169000.0,33.51,1.0,1983.0,97072,WOHNUNG,ZUM_KAUF,0,0,0,...,0,0,0,0,0,0,1,0,0,0
3,275000.0,65.45,2.0,1958.0,97072,WOHNUNG,ZUM_KAUF,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,239000.0,79.50,3.0,1977.0,97209,WOHNUNG,ZUM_KAUF,1,0,0,...,0,0,1,0,0,0,0,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
249,349000.0,71.23,2.0,2019.0,97209,WOHNUNG,ZUM_KAUF,0,0,0,...,0,1,0,0,0,0,0,0,0,0
250,95500.0,31.10,1.0,1997.0,97299,WOHNUNG,ZUM_KAUF,1,0,0,...,0,0,0,0,0,0,0,0,0,1
251,240000.0,97.22,4.0,1991.0,97228,WOHNUNG,ZUM_KAUF,0,0,0,...,0,0,0,1,0,1,0,0,0,1
252,289000.0,75.00,3.0,2010.0,97084,WOHNUNG,ZUM_KAUF,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [149]:
y = data['Object_price']
X = data.drop('Object_price', axis=1)
print(y.head())
print(X.head())

0    220000.0
1    269000.0
2    169000.0
3    275000.0
4    239000.0
Name: Object_price, dtype: float64
   LivingSpace  Rooms  ConstructionYear  ZipCode EstateTypeGerman  \
0        53.00    2.0            1971.0    97080          WOHNUNG   
1        70.00    3.0            1968.0    97078          WOHNUNG   
2        33.51    1.0            1983.0    97072          WOHNUNG   
3        65.45    2.0            1958.0    97072          WOHNUNG   
4        79.50    3.0            1977.0    97209          WOHNUNG   

  DistributionTypeGerman  abstellraum  als_ferienimmobilie_geeignet  \
0               ZUM_KAUF            0                             0   
1               ZUM_KAUF            0                             0   
2               ZUM_KAUF            0                             0   
3               ZUM_KAUF            0                             0   
4               ZUM_KAUF            1                             0   

   altbau_(bis_1945)  bad/wc_getrennt  ...  teppich  

In [150]:
print(data['als_ferienimmobilie_geeignet'])

0      0
1      0
2      0
3      0
4      0
      ..
249    0
250    0
251    0
252    0
253    0
Name: als_ferienimmobilie_geeignet, Length: 254, dtype: int64


In [151]:
train_X, test_X, train_y, test_y = train_test_split(X, y, 
                                                  train_size=0.8, 
                                                  random_state = 0)
train_X, val_X, train_y, val_y = train_test_split(train_X, train_y, 
                                                  train_size=0.8, 
                                                  random_state = 0)

In [152]:
train_X, val_X, train_y, val_y = train_test_split(train_X, train_y, 
                                                  train_size=0.8, 
                                                  random_state = 0)

In [153]:
num_cols = [col for col in train_X.columns if train_X[col].dtype == 'float64']
cat_cols = [col for col in train_X.columns if train_X[col].dtype == 'object']
bin_cols = [col for col in train_X.columns if train_X[col].dtype == 'int64']


In [154]:
print(bin_cols)

['ZipCode', 'abstellraum', 'als_ferienimmobilie_geeignet', 'altbau_(bis_1945)', 'bad/wc_getrennt', 'balkon', 'barriefrei', 'block', 'carport', 'dachgeschoss', 'denkmalgeschuetzt', 'dielen', 'dsl', 'duplex', 'dusche', 'einbauk\\u00fcche', 'elektro', 'erdgeschoss', 'erstbezug', 'estrich', 'etagenheizung', 'fenster', 'fern', 'ferne', 'fliesen', 'fluessiggas', 'frei', 'fu\\u00dfbodenheizung', 'gaestewc', 'garage', 'garten', 'gartennutzung', 'gas', 'gepflegt', 'haustiere_erlaubt', 'holzfenster', 'kable_sat_tv', 'kamera', 'kamin', 'kapitalanlage', 'kelleranteil', 'kfw40', 'kfw55', 'kfw70', 'kontrollierte_be-_und_entl\\u00fcftungsanlage', 'kunststoff', 'kunststofffenster', 'laminat', 'linoleum', 'loggia', 'luftwp', 'massivhaus', 'moebliert', 'neubau', 'neuwertig', 'oel', 'offene_k\\u00fcche', 'pantry', 'parkett', 'pellet', 'personenaufzug', 'provisionsfrei', 'reinigung', 'renoviert', 'renovierungsbed\\u00fcrftig', 'rollstuhlgerecht', 'sat', 'sauna', 'souterrain', 'speisekammer', 'stein', 'ste

In [155]:
encoder = OneHotEncoder(handle_unknown='ignore')
X_train_categorical_xgb = encoder.fit_transform(train_X[cat_cols]).toarray()
X_valid_categorical_xgb = encoder.transform(val_X[cat_cols]).toarray()
X_test_categorical_xgb = encoder.transform(test_X[cat_cols]).toarray()
X_train_processed = np.concatenate([train_X[num_cols], X_train_categorical_xgb], axis=1)
X_valid_processed = np.concatenate([val_X[num_cols], X_valid_categorical_xgb], axis=1)
X_test_processed = np.concatenate([test_X[num_cols], X_test_categorical_xgb], axis=1)
X_train_processed = np.concatenate([train_X[bin_cols], X_train_processed], axis=1)
X_valid_processed = np.concatenate([val_X[bin_cols], X_valid_processed], axis=1)
X_test_processed = np.concatenate([test_X[bin_cols], X_test_processed], axis=1)

In [156]:
X_train_processed[0]

array([9.7072e+04, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00,
       0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00,
       0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00,
       0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00,
       1.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00,
       0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00,
       0.0000e+00, 0.0000e+00, 1.0000e+00, 1.0000e+00, 0.0000e+00,
       0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00,
       0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00,
       0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00,
       0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00,
       0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00,
       0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00,
       0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00,
       0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e

In [158]:
model = xgb.XGBRegressor(eval_metric=['rmse', 'mae'], early_stopping_rounds=15)
model.fit(X=X_train_processed, 
          y=train_y,
          eval_set=[(X_valid_processed, val_y)],  # Liste mit Validierungsdaten und Zielvariablen
          verbose=True
)

preds= model.predict(X_test_processed)
score = mean_absolute_error(test_y, preds)

print("MAE: {}".format(score))

[0]	validation_0-rmse:265347.42742	validation_0-mae:228880.69484
[1]	validation_0-rmse:200584.16698	validation_0-mae:167551.97976
[2]	validation_0-rmse:157156.51821	validation_0-mae:124383.40057
[3]	validation_0-rmse:120072.15198	validation_0-mae:90382.16951
[4]	validation_0-rmse:94238.98342	validation_0-mae:69739.62713
[5]	validation_0-rmse:82409.83093	validation_0-mae:59889.61222
[6]	validation_0-rmse:71848.50921	validation_0-mae:51868.21307


[7]	validation_0-rmse:66368.22040	validation_0-mae:47406.08617
[8]	validation_0-rmse:59916.75696	validation_0-mae:43807.54972
[9]	validation_0-rmse:55618.90120	validation_0-mae:41284.09470
[10]	validation_0-rmse:54081.68102	validation_0-mae:41087.75426
[11]	validation_0-rmse:52114.23086	validation_0-mae:39776.07860
[12]	validation_0-rmse:51061.55728	validation_0-mae:39522.87074
[13]	validation_0-rmse:50537.17539	validation_0-mae:39321.77154
[14]	validation_0-rmse:50369.42995	validation_0-mae:39493.80019
[15]	validation_0-rmse:50190.42447	validation_0-mae:39380.28812
[16]	validation_0-rmse:50290.98167	validation_0-mae:39530.92614
[17]	validation_0-rmse:50115.69449	validation_0-mae:39510.34375
[18]	validation_0-rmse:50407.57102	validation_0-mae:39784.54593
[19]	validation_0-rmse:50387.79371	validation_0-mae:39865.83239
[20]	validation_0-rmse:50624.00217	validation_0-mae:40022.68537
[21]	validation_0-rmse:50787.58455	validation_0-mae:40374.76065
[22]	validation_0-rmse:50888.33398	validati